<a href="https://colab.research.google.com/github/yj5x/AI-tasks/blob/main/Lab_Quiz%E2%80%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit-drawable-canvas

In [ ]:
%%writefile app.py
import torchvision
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image

import torchvision.transforms as transforms
from streamlit_drawable_canvas import st_canvas
import torch.nn as nn
import streamlit as st
import torch.optim as optim

In [ ]:


transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
for images, labels in trainloader:
    break

plt.imshow(images[0].squeeze(), cmap='gray')
plt.axis('off')
plt.title(f"Label: {labels[0].item()}")
plt.show()


In [ ]:
#  CNN

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 8, 3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(8 * 13 * 13, 10)

    def forward(self, x):
        x = self.conv(x)
        x = x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [ ]:
# Model training

model = CNN()
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 10
for epoch in range(epochs):
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%')

In [ ]:

# save the model
torch.save(model.state_dict(), 'mnist_model.pth')
print("model saved successfully")

In [ ]:
# Image processing

def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
    ])
    return 1 - transform(image).unsqueeze(0)  # قلب الألوان فقط

In [ ]:

# App Interface

import streamlit as st
st.title("Draw a number")
st.write('Press the prediction button After you draw')
st.markdown("Draw a number in the box below")

In [ ]:
# drawing interface

canvas_result = st_canvas(
    fill_color="white",
    stroke_width=10,
    stroke_color="white",
    background_color="black",
    width=200,
    height=200,
    drawing_mode="freedraw",
    key="canvas"
)

In [ ]:

if st.button('predict'):
 if canvas_result.image_data is not None:
     image = Image.fromarray((canvas_result.image_data[:, :, 0]).astype(np.uint8))
     resized_image = image.resize((28, 28))

     st.image(resized_image, caption="Resized Input", width=150)

     input_tensor = preprocess_image(resized_image)

     #
     with torch.no_grad():
         output = model(input_tensor)
         pred = torch.argmax(output, dim=1).item()

     st.success(f"Predicted Digit: {pred}")


In [ ]:
###### Prediction


if st.button('Predict'):
    if canvas_result.image_data is not None:
        # تحويل الصورة
        img = Image.fromarray(canvas_result.image_data.astype('uint8')[:, :, 0])
        img = img.resize((28, 28))

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
        img_tensor = transform(img).unsqueeze(0)

        # التنبؤ
        with torch.no_grad():
            output = model(img_tensor)
            _, predicted = torch.max(output.data, 1)

        st.success(f'The expected number is: {predicted.item()}')


if st.button('delete'):
    st.session_state.canvas = np.zeros((200, 200), dtype=np.uint8)
    st.experimental_rerun()

In [ ]:
!pip install streamlit pyngrok



In [ ]:
from pyngrok import ngrok

# ngrok
ngrok.set_auth_token("2zP7EKipfh1OKxPcmvIuWzC0A4j_772E8dHV67mJKCciaMFeV")

# Run Streamlit
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')
# شغل السيرفر على 8501
#!streamlit run model.py --server.port 8501 >/dev/null 2>&1 &

# git the link
public_url = ngrok.connect(8501,'http')
print("the link", public_url)


!streamlit run app.py &